In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

I build this scrit to see wether I can reproduce the electron electron interaction energy form the 2RDM, this serves as test to see wether I understand the data struvctures and theorie accurately to move on to other ideas.

The following function caculates the electron electron interactio  energy from the Fock basis representation of the 2RDM $\gamma_2$ in the follwong way:
$$E_{ee} = \sum_p \sum_q \sum_r \sum_s \gamma_{2_{pqrs}} \langle p q | r s \rangle$$

In [2]:
def ee_from_gamma_2(eri,dm2):
    E = 0 
    dim = eri.shape[0]
    for p in range(0,dim):
        for q in range(0,dim):
            for r in range(0,dim):
                for s in range(0,dim):
                    E+=eri[p,q,r,s]*dm2[p,q,r,s]
    return E

In [3]:
def HF_2RDM(n, M):
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==k and j==l:
                        TWORDM[i,j,k,l] = 1
                    if i==l and j==k:
                        TWORDM[i,j,k,l] -= 1
                    TWORDM[i,j,k,l] = n[i]*n[j]/2*TWORDM[i,j,k,l]
    return TWORDM
    

In [4]:
def FOURIDX_2_NAOS(eri, C):
    for i in range(4):
        Fouridx = np.tensordot(eri, C, axes=1).transpose(3, 0, 1, 2)
    return Fouridx

In [5]:
def print_2RDM(TWORDM):
    M = TWORDM.shape[0]
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    print(i,j,k,l, TWORDM[i,j,k,l]) 

In [6]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
#mol.basis = "cc-pvdz"
mol.basis = "6-31g" 
mol.build()
print(mol.nelec)

(1, 1)


In [7]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()
h = mf.get_hcore()
C = mf.mo_coeff
h1 = C.T@h@C

converged SCF energy = -2.85516042615445


In [8]:
# Find electron-repulsion integrals (eri).
eri = ao2mo.kernel(mol, mf.mo_coeff)
eri = np.asarray(ao2mo.restore(1, eri, mol.nao))

In [9]:
print(eri.shape)

(2, 2, 2, 2)


In [10]:
# First create FCI solver with function fci.FCI and solve the FCI problem
#
cisolver = fci.FCI(mol, mf.mo_coeff)
e, fcivec = cisolver.kernel()

In [11]:
dm1, dm2 = cisolver.make_rdm12(fcivec, mf.mo_coeff.shape[1], mol.nelec)

In [12]:
print(.5*ee_from_gamma_2(eri,dm2), e-np.trace(np.matmul(dm1,h1)))

0.9934201474038266 0.9934201474038242


In [13]:
FCIoccu, FCInaturalC = np.linalg.eigh(dm1)
hf_dm2 = HF_2RDM(FCIoccu, h.shape[0])
h2_NAO = FOURIDX_2_NAOS(eri, FCInaturalC)

In [14]:
print(.5*ee_from_gamma_2(h2_NAO,hf_dm2), e-np.trace(np.matmul(dm1,h1)))
print(hf_dm2, "\n", dm2)

-2.168404344971009e-19 0.9934201474038242
[[[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.00860964]
   [-0.00860964  0.        ]]]


 [[[ 0.         -0.00860964]
   [ 0.00860964  0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]] 
 [[[[ 1.99133674e+00  4.15105995e-03]
   [ 4.15105995e-03  8.65313150e-06]]

  [[ 4.15105995e-03 -1.31213584e-01]
   [ 8.65313150e-06 -2.73522526e-04]]]


 [[[ 4.15105995e-03  8.65313150e-06]
   [-1.31213584e-01 -2.73522526e-04]]

  [[ 8.65313150e-06 -2.73522526e-04]
   [-2.73522526e-04  8.64595344e-03]]]]


In [15]:
print_2RDM(dm2)

0 0 0 0 1.9913367402938411
0 0 0 1 0.004151059946534481
0 0 1 0 0.004151059946534481
0 0 1 1 8.653131502600713e-06
0 1 0 0 0.004151059946534481
0 1 0 1 -0.13121358445764078
0 1 1 0 8.653131502634262e-06
0 1 1 1 -0.00027352252577982365
1 0 0 0 0.004151059946534481
1 0 0 1 8.653131502634262e-06
1 0 1 0 -0.13121358445764078
1 0 1 1 -0.00027352252577982365
1 1 0 0 8.653131502600713e-06
1 1 0 1 -0.00027352252577982365
1 1 1 0 -0.00027352252577982365
1 1 1 1 0.008645953443153914
